In [ ]:
import os
import json
import h5py
from data_loader import ImageSequence

def dict_to_json_serializable(d):
    """Recursively convert non-serializable values to strings"""
    def _convert(value):
        if isinstance(value, (str, int, float, bool, type(None))):
            return value
        elif isinstance(value, dict):
            return {k: _convert(v) for k, v in value.items()}
        elif isinstance(value, (list, tuple)):
            return [_convert(v) for v in value]
        else:
            return str(value)  # Convert non-serializable objects to strings
    return _convert(d)

def save_stack_hdf5(directory, output_file="stack_testing.h5"):
    with h5py.File(os.path.join(directory, output_file), "w") as hf:
        img_group = hf.create_group("images")
        meta_group = hf.create_group("metadata")

        valid_files = sorted([
            f for f in os.listdir(directory) 
            if f.endswith('.ndata1') or f.endswith('tif') and 'SuperScan (HAADF) (Gaussian Blur)' not in f
        and 'EELS' not in f])

        for idx, filename in enumerate(valid_files):
            file_path = os.path.join(directory, filename)
            
            # Load image data
            imageseq = ImageSequence(file_path)
            img = imageseq.raw_data
            
            # Store image with compression
            img_ds = img_group.create_dataset(
                name=f"image_{idx:04d}",
                data=img,
                compression="gzip"
            )
            
            # Handle metadata with fallback
            try:
                raw_meta = imageseq.raw_metadata or {}
            except AttributeError:
                raw_meta = {}
                
            # Convert to JSON-serializable format
            processed_meta = dict_to_json_serializable(raw_meta)
            
            # Add fallback if empty
            if not processed_meta:
                processed_meta = {"metadata_status": "no_metadata_found"}
                
            # Store metadata as JSON string
            meta_ds = meta_group.create_dataset(
                name=f"metadata_{idx:04d}",
                data=json.dumps(processed_meta),
                dtype=h5py.string_dtype()
            )
            
            # Create cross-reference
            img_ds.attrs["metadata_ref"] = f"metadata_{idx:04d}"
            meta_ds.attrs["image_ref"] = f"image_{idx:04d}"

def process_directory(directory):
    if any(f.endswith('.ndata1') or f.endswith('.tif') for f in os.listdir(directory)):
        save_stack_hdf5(directory)
    else:
        for subdir in os.listdir(directory):
            subdir_path = os.path.join(directory, subdir)
            if os.path.isdir(subdir_path):
                process_directory(subdir_path)

if __name__ == "__main__":
    process_directory('/home/somar/Desktop/2025/Data for publication/Sample 2344/ADF images/')
    # process_directory('/home/somar/Desktop/2025/Data for publication/Multilayer graphene/')
    process_directory('/home/somar/Downloads/Schrirang/')


In [ ]:
import os
import json
import h5py
from data_loader import ImageSequence

def dict_to_json_serializable(d):
    def _convert(value):
        if isinstance(value, (str, int, float, bool, type(None))):
            return value
        elif isinstance(value, dict):
            return {k: _convert(v) for k, v in value.items()}
        elif isinstance(value, (list, tuple)):
            return [_convert(v) for v in value]
        else:
            return str(value)
    return _convert(d)

def save_stack_hdf5(directory, output_file="stack.h5"):
    valid_files = sorted([
        f for f in os.listdir(directory)
        if f.endswith('.ndata1') and 'SuperScan (HAADF) (Gaussian Blur)' not in f
    ])

    vcr_files = []
    non_vcr_files = []

    for filename in valid_files:
        file_path = os.path.join(directory, filename)
        imageseq = ImageSequence(file_path)
        if imageseq.raw_data.ndim == 3:
            vcr_files.append((filename, imageseq))
        else:
            non_vcr_files.append((filename, imageseq))

    if non_vcr_files:
        with h5py.File(os.path.join(directory, output_file), "w") as hf:
            img_group = hf.create_group("images")
            meta_group = hf.create_group("metadata")

            fov_data = []
            for idx, (filename, imageseq) in enumerate(non_vcr_files):
                try:
                    raw_meta = imageseq.raw_metadata or {}
                except AttributeError:
                    raw_meta = {}
                processed_meta = dict_to_json_serializable(raw_meta)
                metadata = processed_meta.get('metadata', {})

                fov = float('inf')
                try:
                    fov = float(metadata.get('scan', {}).get('scan_device_properties', {}).get('fov_nm', float('inf')))
                except KeyError:
                    try:
                        fov = float(processed_meta.get('instrument', {}).get('ImageScanned', {}).get('fov_nm', float('inf')))
                    except KeyError:
                        pass

                fov_data.append((idx, fov, imageseq.raw_data, processed_meta))

            fov_data.sort(key=lambda x: x[1])

            for new_idx, (orig_idx, fov, img, meta) in enumerate(fov_data):
                img_ds = img_group.create_dataset(
                    f"image_{new_idx:04d}",
                    data=img,
                    compression="gzip"
                )
                img_ds.attrs["fov_nm"] = fov if fov != float('inf') else "unknown"

                meta_ds = meta_group.create_dataset(
                    f"metadata_{new_idx:04d}",
                    data=json.dumps(meta),
                    dtype=h5py.string_dtype()
                )
                img_ds.attrs["metadata_ref"] = f"metadata_{new_idx:04d}"
                meta_ds.attrs["image_ref"] = f"image_{new_idx:04d}"
                meta_ds.attrs["original_index"] = orig_idx

            hf.attrs["sorting"] = "fov_nm (unknown FOV at end)"
            hf.attrs["sorting_version"] = "1.0"

    for vcr_idx, (filename, imageseq) in enumerate(vcr_files, start=1):
            vcr_output = os.path.join(directory, f"VCR{vcr_idx}.h5")
            with h5py.File(vcr_output, "w") as vcr_hf:
                img_group = vcr_hf.create_group("images")
                meta_group = vcr_hf.create_group("metadata")

                # Get raw metadata
                try:
                    raw_meta = imageseq.raw_metadata or {}
                except AttributeError:
                    raw_meta = {}
                processed_meta = dict_to_json_serializable(raw_meta)
                metadata = processed_meta.get('metadata', {})
                # Extract timeseries data
                timeseries = metadata.get('hardware_source', {}).get('timeseries', [])
                img_data = imageseq.raw_data
                num_images = img_data.shape[0]

                # Create timestamp-FOV mapping
                fov_map = []
                for ts_entry in timeseries:
                    fov_map.append((
                        ts_entry.get('timestamp', float('inf')),
                        ts_entry.get('FOV', float('inf'))
                    ))

                for i in range(num_images):
                    # Clone base metadata
                    image_meta = metadata.copy()
                    
                    # Find closest timestamp match
                    if i < len(fov_map):
                        # Direct index match if available
                        ts, fov = fov_map[i]
                        print(f"Using field of view {fov} for image {i}")
                        metadata['scan']['scan_device_properties']['fov_nm'] = fov

                    else:
                        # Fallback: find nearest timestamp
                        ts, fov = min(fov_map, key=lambda x: abs(x[0] - (fov_map[-1][0] if fov_map else 0)))
                        
                    # Update metadata with time-specific FOV
                    image_meta['scan'] = image_meta.get('scan', {})
                    image_meta['scan']['scan_device_properties'] = image_meta['scan'].get('scan_device_properties', {})
                    image_meta['scan']['scan_device_properties']['fov_nm'] = fov

                    # Store metadata
                    meta_ds = meta_group.create_dataset(
                        name=f"metadata_{i:04d}",
                        data=json.dumps(image_meta),
                        dtype=h5py.string_dtype()
                    )

                    # Store image
                    img_slice = img_data[i]
                    img_ds = img_group.create_dataset(
                        f"image_{i:04d}",
                        data=img_slice,
                        compression="gzip"
                    )

                    # Link metadata
                    img_ds.attrs["metadata_ref"] = f"metadata_{i:04d}"
                    meta_ds.attrs["image_ref"] = f"image_{i:04d}"
                    meta_ds.attrs["timestamp"] = ts
                    meta_ds.attrs["fov_nm"] = fov

                # Add validation attributes
                vcr_hf.attrs.update({
                    "is_vcr_stack": True,
                    "num_images": num_images,
                    "timeseries_entries": len(timeseries),
                    "fov_mapping": "timestamp-matched" if len(timeseries) >= num_images else "fallback",
                    "original_filename": filename
                })



def process_directory(directory):
    if any(f.endswith('.ndata1') for f in os.listdir(directory)):
        save_stack_hdf5(directory)
    else:
        for subdir in os.listdir(directory):
            subdir_path = os.path.join(directory, subdir)
            if os.path.isdir(subdir_path):
                process_directory(subdir_path)

if __name__ == "__main__":
    process_directory('/home/somar/lab course/')

In [1]:
import os
import json
import h5py
from data_loader import ImageSequence
def dict_to_json_serializable(d):
    """Recursively convert non-serializable values to strings"""
    def _convert(value):
        if isinstance(value, (str, int, float, bool, type(None))):
            return value
        elif isinstance(value, dict):
            return {k: _convert(v) for k, v in value.items()}
        elif isinstance(value, (list, tuple)):
            return [_convert(v) for v in value]
        else:
            return str(value)  # Convert non-serializable objects to strings
    return _convert(d)

def save_stack_hdf5(directory, output_file="stacktest.h5"):
    with h5py.File(os.path.join(directory, output_file), "w") as hf:
        img_group = hf.create_group("images")
        meta_group = hf.create_group("metadata")

        valid_files = sorted([
            f for f in os.listdir(directory) 
            if f.endswith('.ndata1') and 'SuperScan (HAADF) (Gaussian Blur)' not in f
        ])

        fov_data = []

        for idx, filename in enumerate(valid_files):
            file_path = os.path.join(directory, filename)
            imageseq = ImageSequence(file_path)

            try:
                raw_meta = imageseq.raw_metadata or {}
            except AttributeError:
                raw_meta = {}

            try:
                metadata_dict = raw_meta[0] if hasattr(raw_meta, '__getitem__') else raw_meta
            except Exception:
                metadata_dict = {}

            metadata = metadata_dict.get('metadata', {})

            # Extract FOV with fallbacks
            fov = float('inf')
            try:
                fov = float(metadata.get('scan', {}).get('scan_device_properties', {}).get('fov_nm', float('inf')))
            except Exception:
                try:
                    fov = float(metadata_dict.get('instrument', {}).get('ImageScanned', {}).get('fov_nm', float('inf')))
                except Exception:
                    pass


            try:
                img = imageseq.raw_data[0] 
            except Exception:
                continue

            fov_data.append((idx, fov, img, metadata_dict))

        # Sort by FOV (lowest first; unknown at end)
        fov_data.sort(key=lambda x: x[1])

        for new_idx, (orig_idx, fov, img, meta) in enumerate(fov_data):
            # Store image
            img_ds = img_group.create_dataset(
                name=f"image_{new_idx:04d}",
                data=img,
                compression="gzip"
            )
            img_ds.attrs["fov_nm"] = fov if fov != float('inf') else "unknown"

            # Store metadata
            meta_ds = meta_group.create_dataset(
                name=f"metadata_{new_idx:04d}",
                data=json.dumps(dict_to_json_serializable(meta)),
                dtype=h5py.string_dtype()
            )

            # Cross-reference
            img_ds.attrs["metadata_ref"] = f"metadata_{new_idx:04d}"
            meta_ds.attrs["image_ref"] = f"image_{new_idx:04d}"
            meta_ds.attrs["original_index"] = orig_idx

        hf.attrs["sorting"] = "fov_nm (unknown FOV at end)"
        hf.attrs["sorting_version"] = "1.0"




def process_directory(directory):
    if any(f.endswith('.ndata1') for f in os.listdir(directory)):
        save_stack_hdf5(directory)
    else:
        for subdir in os.listdir(directory):
            subdir_path = os.path.join(directory, subdir)
            if os.path.isdir(subdir_path):
                process_directory(subdir_path)


if __name__ == "__main__":
    # process_directory('/home/somar/Desktop/2025/Data for publication/')
    process_directory('/home/somar/Desktop/2025/Data for publication/Multilayer graphene/')
    # process_directory('/home/somar/Desktop/2025/Data for publication/Sample 2525/SSB reconstruction of 4d STEM data/')
    # process_directory('/home/somar/Desktop/2025/Data for publication/Sample 2525/ADF images/')
    # process_directory('/home/somar/test /')
    process_directory('/home/somar/lab course/')


In [ ]:
import os
import h5py
import json
from imagesequence import ImageSequence

def serialize_metadata(meta):
    """Recursively converts metadata to JSON-serializable format"""
    def _serialize(obj):
        # Handles basic types
        if isinstance(obj, (str, int, float, bool, type(None))):
            return obj
        # Recursively process dictionaries
        elif isinstance(obj, dict):
            return {k: _serialize(v) for k, v in obj.items()}
        # Process lists/tuples
        elif isinstance(obj, (list, tuple)):
            return [_serialize(v) for v in obj]
        # Convert other types to strings
        else:
            return str(obj)
    return _serialize(meta)

def save_stack_hdf5(directory, output_file="stack.h5"):
    """Main function to create HDF5 stack"""
    with h5py.File(os.path.join(directory, output_file), "w") as hf:
        # Get sorted list of valid files
        valid_files = sorted([
            f for f in os.listdir(directory)
            if f.endswith('.ndata1') and 'SuperScan (HAADF) (Gaussian Blur)' not in f
        ])

        for idx, filename in enumerate(valid_files):
            # Create unique group for each image
            img_group = hf.create_group(f"image_{idx:04d}")
            
            # Load and store image data
            imageseq = ImageSequence(os.path.join(directory, filename))
            img_group.create_dataset("data", 
                                   data=imageseq.raw_data, 
                                   compression="gzip")
            
            # Handle metadata
            try:
                raw_meta = imageseq.raw_metadata or {}
            except AttributeError:
                raw_meta = {"metadata_status": "no_metadata_found"}
            
            # Store processed metadata as JSON string
            processed_meta = serialize_metadata(raw_meta)
            img_group.attrs.update({
                "metadata": json.dumps(processed_meta),
                "original_filename": filename
            })

def process_directory(directory):
    """Recursive directory processor"""
    if any(f.endswith('.ndata1') for f in os.listdir(directory)):
        save_stack_hdf5(directory)
    else:
        for subdir in os.listdir(directory):
            subdir_path = os.path.join(directory, subdir)
            if os.path.isdir(subdir_path):
                process_directory(subdir_path)



def load_image_with_metadata(h5_file, index):
    with h5py.File(h5_file, "r") as hf:
        img = hf[f"images/image_{index:04d}"][:]
        meta_str = hf[f"metadata/metadata_{index:04d}"][()]
        metadata = json.loads(meta_str)
        return img, metadata
# Example usage:
img, metadata = load_image_with_metadata("stack.h5", 0)
img, metadata


def load_full_stack(h5_file):
    images = []
    metadatas = []
    with h5py.File(h5_file, "r") as hf:
        # Get sorted group names
        data_group = hf["images"]
        groups = sorted([k for k in data_group if k.startswith("image_")], 
                       key=lambda x: int(x.split("_")[1]))
        print(groups)
        metadata_group= hf[f"metadata"]
        metadata = sorted([k for k in metadata_group if k.startswith("metadata_")], 
                       key=lambda x: int(x.split("_")[1]))
        
        for group_name in groups:
            # print(f"images/{group_name}")
            group = hf[f"images/{group_name}"]
            images.append(group[:])

        for group_name in metadata:
            # print(f"metadata/{group_name}")
            group = hf[f"metadata/{group_name}"]
            metadatas.append(json.loads(group[()]))
    return images, metadatas
# Example usage
h5_file = "stack.h5"
images , metadata= load_full_stack(h5_file)
len(images), len(metadata)


In [ ]:
import h5py
import json

def load_image_with_metadata(h5_file, index):
    with h5py.File(h5_file, "r") as hf:
        img = hf[f"images/image_{index:04d}"][:]
        meta_str = hf[f"metadata/metadata_{index:04d}"][()]
        metadata = json.loads(meta_str)
        return img, metadata
# Example usage:
img, metadata = load_image_with_metadata("stack.h5", 0)
img, metadata


In [ ]:
import h5py
import json
def load_full_stack(h5_file):
    images = []
    metadatas = []
    with h5py.File(h5_file, "r") as hf:
        # Get sorted group names
        data_group = hf["images"]
        groups = sorted([k for k in data_group if k.startswith("image_")], 
                       key=lambda x: int(x.split("_")[1]))
        print(groups)
        metadata_group= hf[f"metadata"]
        metadata = sorted([k for k in metadata_group if k.startswith("metadata_")], 
                       key=lambda x: int(x.split("_")[1]))
        
        for group_name in groups:
            # print(f"images/{group_name}")
            group = hf[f"images/{group_name}"]
            images.append(group[:])

        for group_name in metadata:
            # print(f"metadata/{group_name}")
            group = hf[f"metadata/{group_name}"]
            metadatas.append(json.loads(group[()]))
    return images, metadatas
# Example usage
h5_file = "stack.h5"
images , metadata= load_full_stack(h5_file)
len(images), len(metadata)

In [ ]:
import os
import json
import h5py
import numpy as np
from imagesequence import ImageSequence

def serialize_metadata(meta):
    """Recursively convert metadata to JSON-serializable format"""
    def _serialize(obj):
        if isinstance(obj, (str, int, float, bool, type(None))):
            return obj
        elif isinstance(obj, dict):
            return {k: _serialize(v) for k, v in obj.items()}
        elif isinstance(obj, (list, tuple)):
            return [_serialize(v) for v in obj]
        else:
            return str(obj)
    return _serialize(meta)

def save_stack_hdf5(directory, output_file="fast_stack.h5"):
    with h5py.File(os.path.join(directory, output_file), "w") as hf:
        # Get sorted list of valid files
        valid_files = sorted([
            f for f in os.listdir(directory)
            if f.endswith('.ndata1') and 'SuperScan (HAADF) (Gaussian Blur)' not in f
        ])
        num_images = len(valid_files)

        # Create compound datatype
        dt = np.dtype([
            ('image', h5py.vlen_dtype(np.float32)),  # Adjust dtype as needed
            ('metadata', h5py.string_dtype()),
            ('shape_0', np.int32),
            ('shape_1', np.int32)
        ])

        # Create single dataset for all images
        ds = hf.create_dataset("image_stack", (num_images,), 
                             dtype=dt, compression="gzip")

        for idx, filename in enumerate(valid_files):
            # Load data
            imageseq = ImageSequence(os.path.join(directory, filename))
            img = imageseq.raw_data.astype(np.float32)
            orig_shape = img.shape
            
            # Process metadata
            try:
                meta = serialize_metadata(imageseq.raw_metadata)
            except AttributeError:
                meta = {"status": "no_metadata"}
            
            # Store in compound dataset
            ds[idx] = (
                img.flatten(),  # vlen array
                json.dumps(meta),
                orig_shape[0],
                orig_shape[1]
            )

def load_full_stack(h5_file):
    """Load all data in ~2x faster than group-based approach"""
    with h5py.File(h5_file, "r") as hf:
        ds = hf["image_stack"]
        # Single read operation for all data
        all_data = ds[:]
        
    # Reconstruct images and metadata
    images = [
        arr.reshape((shape0, shape1)) 
        for arr, _, shape0, shape1 in all_data
    ]
    metadata = [
        json.loads(meta) 
        for _, meta, _, _ in all_data
    ]
    return images, metadata


def process_directory(directory):
    """Recursive directory processor"""
    if any(f.endswith('.ndata1') for f in os.listdir(directory)):
        save_stack_hdf5(directory)
    else:
        for subdir in os.listdir(directory):
            subdir_path = os.path.join(directory, subdir)
            if os.path.isdir(subdir_path):
                process_directory(subdir_path)

if __name__ == "__main__":
    # process_directory('/home/somar/Desktop/2025/Data for publication/Multilayer graphene/')
    img, meta = load_full_stack("fast_stack.h5")
    print(img[100].shape, meta[100])


In [ ]:
# Conversion script (PKL -> HDF5)
import h5py
import json
import numpy as np
from imagesequence import ImageSequence


stacks = ['/home/somar/Desktop/2025/Data for publication/Sample 2344/ADF images/2024-12-19_After_cleaning_280C_2h/stack.pkl',
            '/home/somar/Desktop/2025/Data for publication/Sample 2344/ADF images/2024-12-19_After_Plasma_irradiation/stack.pkl',
            '/home/somar/Desktop/2025/Data for publication/Sample 2344/ADF images/2024-12-23_After_LaserCleaning_PtEvaporation/stack.pkl',
            '/home/somar/Desktop/2025/Data for publication/Sample 2344/ADF images/After_Heating_200C/2025-01-08_After_Heating_200C/stack.pkl',
            '/home/somar/Desktop/2025/Data for publication/Sample 2344/ADF images/After_Heating_200C/2025-01-20_After_Heating_200C/stack.pkl',
            '/home/somar/Desktop/2025/Data for publication/Sample 2344/ADF images/After_Heating_150C/2025-02-06_After_Heating_150C/stack.pkl']

metadata_s= ['/home/somar/Desktop/2025/Data for publication/Sample 2344/ADF images/2024-12-19_After_cleaning_280C_2h/metadata.pkl',
            '/home/somar/Desktop/2025/Data for publication/Sample 2344/ADF images/2024-12-19_After_Plasma_irradiation/metadata.pkl',
            '/home/somar/Desktop/2025/Data for publication/Sample 2344/ADF images/2024-12-23_After_LaserCleaning_PtEvaporation/metadata.pkl',
            '/home/somar/Desktop/2025/Data for publication/Sample 2344/ADF images/After_Heating_200C/2025-01-08_After_Heating_200C/metadata.pkl',
            '/home/somar/Desktop/2025/Data for publication/Sample 2344/ADF images/After_Heating_200C/2025-01-20_After_Heating_200C/metadata.pkl',
            '/home/somar/Desktop/2025/Data for publication/Sample 2344/ADF images/After_Heating_150C/2025-02-06_After_Heating_150C/metadata.pkl']







In [ ]:
class H5ImageSequence:
    def __init__(self, h5_file):
        self.h5_file = h5_file
        self._file = None  # Keep file handle open
        
    def __enter__(self):
        self._file = h5py.File(self.h5_file, 'r')
        return self
    
    def __exit__(self):
        if self._file:
            self._file.close()
            
    @property
    def raw_data(self):
        return H5LazyLoader(self.h5_file, 'images')
    
    @property
    def raw_metadata(self):
        return H5LazyLoader(self.h5_file, 'metadata')






class H5LazyLoader:
    def __init__(self, h5_file, group):
        self.h5_file = h5_file
        self.group = group
        
    def __getitem__(self, index):
        with h5py.File(self.h5_file, 'r') as hf:
            if self.group == 'images':
                return hf[self.group][f"image_{index:04d}"][:]
            else:  # metadata
                return json.loads(hf[self.group][index].asstr()[()])
    
    def __len__(self):
        with h5py.File(self.h5_file, 'r') as hf:
            return len(hf[self.group])
        
    def get_metadata(self, target_key, data=None):
        """This method gets all the values of a certain target key in the metadata json file."""
        extracted_values = []
        if data is None:
            data = self.raw_metadata
        if isinstance(data, dict):
            if target_key in data and data[target_key] is not None:
                value_to_extract = data.get(target_key)
                extracted_values.append(value_to_extract)
            for key, value in data.items():
                if isinstance(value, (list, dict)):
                    extracted_values.extend(self.get_metadata(target_key, value))
        elif isinstance(data, list):
            for item in data:
                extracted_values.extend(self.get_metadata(target_key, item))
        
        if len(extracted_values) == 1 and isinstance(extracted_values[0], (list)):
            return extracted_values[0]
        return extracted_values
    

    def get_specific_metadata(self, target_key, required_keys=None, data=None, under_required_keys=False):
        """
        This method gets specific metadata based on a condition that the target key is extracted only 
        when under specific required keys in the nested dictionaries. When providing required keys that are 
        at the same branch of the nested dictionary, the method will extract the target key for the one at higher level.
        So either provide required keys that are at different branches or provide one required key.
        """
        if data is None:
            data = self.raw_metadata
        if required_keys is None:
            extracted_values = self.get_metadata(target_key, data)
        else:
            extracted_values = []
            if isinstance(data, dict):
                for key, value in data.items():
                    if key in required_keys:
                        # We are entering a required key scope
                        if isinstance(value, dict):
                            extracted_values.extend(self.get_specific_metadata(target_key, required_keys, value, True))
                        elif isinstance(value, list):
                            for item in value:
                                extracted_values.extend(self.get_specific_metadata(target_key, required_keys, item, True))
                    elif under_required_keys and key == target_key:
                        extracted_values.append(value)
                    else:
                        if isinstance(value, (dict, list)):
                            extracted_values.extend(self.get_specific_metadata(target_key, required_keys, value, under_required_keys))

            elif isinstance(data, list):
                for item in data:
                    extracted_values.extend(self.get_specific_metadata(target_key, required_keys, item, under_required_keys))

        if len(extracted_values) == 1 and isinstance(extracted_values[0], (list)):
            return extracted_values[0]
        return extracted_values


In [ ]:
import os
import json
import h5py
from imagesequence import ImageSequence, H5ImageSequence
# Example usage
stack_h5 = '/home/somar/Desktop/own_stuff/imageanalysis/imageanalysis/src/stack.h5'
stack = H5ImageSequence(stack_h5)
metadata = stack.raw_metadata  # Shared instance for H5 files
indices = [0, 1, 2, 3, 4, 5]  # Example indices to load
data = stack.raw_data  # Shared instance for H5 files
meta = metadata[f"metadata_{1:04d}"]
print(meta)
metadata.get_specific_metadata('fov_nm', required_keys=['scan_device_parameters'], data=meta)
print(data[0])

In [ ]:
%matplotlib qt5
from image_analysis_lastversion03_04_2025 import InteractiveImageAnalysis
directories = ['/home/somar/Desktop/2025/Data for publication/Sample 2438/ADF images',
               '/home/somar/Desktop/2025/Data for publication/Sample 2473/ADF images',
               '/home/somar/Desktop/2025/Data for publication/Sample 2474/ADF images',
               '/home/somar/Desktop/2025/Data for publication/Sample 2475/ADF images']

# Usage example
stacks_path = [stack + '/stack.pkl' for stack in directories][1]
metadata_path = [stack + '/metadata.pkl' for stack in directories][1]
font_path = "/home/somar/.fonts/SourceSansPro-Semibold.otf" 
stack_h5 = '/home/somar/Desktop/own_stuff/imageanalysis/imageanalysis/src/stack.h5'
stack_h5_1 = '/home/somar/Desktop/2025/Data for publication/Sample 2344/ADF images/2024-12-23_After_LaserCleaning_PtEvaporation/stack.h5'
image_analysis = InteractiveImageAnalysis(stack_h5_1, metadata_path=None, analysing_features=True, save_images_with_calibrated_scalebar=False, 
                                        clean_graphene_analysis=True, contamination_analysis=False, fixed_length_scalebar=True, font_path=font_path, 
                                        clusters_analysis=True, defects_analysis=False)

    # This version is based on the display_image_with_scale_bar.ipynb notebook coppied at 03-04-2025 at 4:16

In [ ]:
from imagesequence import ImageSequence
stack_h5 = '/home/somar/Desktop/own_stuff/imageanalysis/imageanalysis/src/stack.h5'
image = ImageSequence(stack_h5)
# image.raw_data
len(image.raw_metadata)

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.read_csv('tst.csv')